

Importação do código de verificação do git



In [1]:
%%capture
!gdown --id '1g-hN06T71k5L_CMj6fMnTCEOPfyFY--M'
!gdown --id '1AROvzRcG54Zt6iMT1-0JNbZg_68Z-jkr'
!gdown --id '1PELvQf76Zzjsda4TA5B_8X-nwUvwxnr7'
!gdown --id '1HcusKPkH__dxHjUj3rqtMsa5hH863npb'
!gdown --id '1pujGP0MUoeljKMHKJpEG2ZI59j-f8MiJ'
!pip install git+https://github.com/grading/gradememaybe.git

# Robô de Dois Elos

Considere um robô de dois elos, conforme ilustra a figura:

<img src="https://drive.google.com/uc?id=1n9_mJ9sj3O2Kp9FOL8gP3RMxHiKK9tRC" width="300">

Considere ainda as equações que definem a posição da garra $p$:

$p = \left[ \begin{array}\\ p_x \\ p_y \end{array}\right] = \left[ \begin{array}\\ l_1 cos(\theta_1) + l_2cos (\theta_1+\theta_2) \\ l_1 sin(\theta_1) + l_2 sin(\theta_1+\theta_2) \\ \end{array} \right]$

E por fim considere o ponto alvo:
$m = \left[\begin{array}\\ m_x\\ m_y\\ \end{array}\right]$.

A distância Euclideana entre $p$ e $m$ é dada por:
$d = || m - p ||_2 = \sqrt{(m_x-p_x)^2 + (m_y-p_y)^2}$

Assim podemos definir uma função de custo $f(\theta_1, \theta_2) = d^2$, que resulta na expressão:
$f(\theta_1,\theta_2)=(m_x-l_1 cos(\theta_1) - l_2cos(\theta_1+\theta_2))^2 + (m_y-l_1 sin(\theta_1) - l_2 sin(\theta_1+\theta_2))^2$

Levando em conta o problema formulado acima:

1. Escreva uma função que, dados os ângulos das juntas $\theta_1$, $\theta_2$, e constantes $l_1$, $l_2$, calcule a posição do ponto $p$.

2. Escreva uma função que, dados o ponto $m$ e os ângulos $\theta_1$ e $\theta_2$ e constantes $l_1$ e $l_2$, calcule a função de custo exatamente como descrita acima.

3. Calcule de forma analítica o gradiente dessa função de custo, com respeito aos ângulos $\theta_1$ e $\theta_2$. Implemente uma função que, dada a pose atual do robô, calcule como saída esse gradiente.

5. Implemente o método da descida do gradiente utilizando as funções implementadas nos itens 2 e 3 acima.

6. Mostre graficamente, numa animação, como o robô chega ao ponto desejado utilizando este método.

## Avaliação

Este código inclui funções de avaliação que testam a sua implementação e dão algum feedback, ainda que limitado. Para pontuar nos itens 1 a 4 acima, você deve garantir que seu código passe nos respectivos testes. Para pontuar no item 6, você deverá observar o robô chegando ao alvo na animação gerada automaticamente pelo código da última célula ao final do documento. Isso deve acontecer automaticamente se você cumpriu todos itens e passou todos testes.

## Implementação

In [ ]:
# Módulo para avaliação automática do exercício
from gofer import ok

# Módulo que simula o robô de dois elos
from robot2link import Robot

# Provavelmente vamos precisar também do
import numpy as np


### Classe principal

Preencha seu código na forma de métodos, nas lacunas deixadas no código da classe abaixo.

In [ ]:
class MyRobot(Robot):

  def __init__(self, **kwargs):
    super().__init__(**kwargs)

  # Retorna a posição do orção terminal 
  def pos(self):
    
    px = np.cos(self.theta1)*self.l1 + np.cos(self.theta1+self.theta2)*self.l2 
    py = np.sin(self.theta1)*self.l1 + np.sin(self.theta1+self.theta2)*self.l2
    
    return np.array([px, py]) 


  # Custo igual a distancia euclidiana de P em relação a M 
  def cost(self):

    px = self.l1*np.cos(self.theta1) + self.l2*np.cos(self.theta1 + self.theta2)
    py = self.l1*np.sin(self.theta1) + self.l2*np.sin(self.theta1 + self.theta2)

    d2 = ((self.mx - px)**2 + (self.my - py)**2)
    return d2


  # Calcula o gradiente da função posição terminal 
  def grad(self):

    h = 0.0000000000000000000001

    px = lambda t1, t2 : self.l1*np.cos(t1) + self.l2*np.cos(t1 + t2)
    py = lambda t1, t2 : self.l1*np.sin(t1) + self.l2*np.sin(t1 + t2)

    f = ((self.mx - px(self.theta1, self.theta2))**2 + (self.my - py(self.theta1, self.theta2))**2)

    dx = (((self.mx - px(self.theta1+h, self.theta2))**2 + (self.my - py(self.theta1+h, self.theta2))**2) - f)/h
    dy = (((self.mx - px(self.theta1, self.theta2+h))**2 + (self.my - py(self.theta1, self.theta2+h))**2) - f)/h

    return np.array([dx, dy])
    
  # Dá um passo em direção ao vale da função de custo
  def optimize(self, step):
    
    gx, gy = self.grad()

    self.theta1 = self.theta1 - step*gx 
    self.theta2 = self.theta2 - step*gy 

    self.move(self.theta1, self.theta2)


O código abaixo ilustra como você pode usar seu robô. Sinta-se à vontade para experimentar e testar colocando novas células de código abaixo.

In [ ]:
# Exemplo de uso
r = MyRobot()
r.set_goal(110,-30)

t0s = np.linspace(0,np.pi,100)
t1s = np.linspace(0,np.pi/2,100)

for t0, t1 in zip(t0s, t1s):
  r.move(t0,t1)
  
r.animation()

# Avaliação automática

Não modifique o código das células a partir desta aqui. Os comandos abaixo geram uma avaliação automática do seu código, oferecendo retorno que pode trazer alguma dica sobre erros mais comuns.

## 1. Cinemática direta

In [ ]:
ok.check('e04_1.py')

## 2. Função de custo

In [ ]:
ok.check('e04_2.py')

## 3. Gradiente

In [ ]:
ok.check('e04_3.py')

## 4. Algoritmo

In [ ]:
ok.check('e04_4.py')

# Resultado final

Quando seu código final passar em todos os testes, o código abaixo irá gerar uma animação do robô chegando ao alvo. Você pode rodar essa célula várias vezes para gerar alvos diferentes.

In [ ]:
# Abaixo criamos um robô e geramos um objetivo
# de coordenadas mx, my aleatórias entre
# -150 e +150. Depois disso rodamos 200
# passos de tamanho 5e-6 corrigindo os
# valores dos ângulos no sentido oposto
# do gradiente.
#
# No resultado a animação deve mostrar
# o robô atingindo o alvo na maioria
# das vezes (em alguns casos 200 passos
# pode ser pouco).
#
# Você pode rodar essa célula diversas
# vezes para ver o robô alcançando diferentes
# objetivos -- se ver isso, significa que você
# também alcançou o seu nesse exercício :-)

r = MyRobot()
mx, my = np.random.rand(2)*300.0 - 150.0
r.set_goal(mx, my)

for _ in range(200):
  r.optimize(0.00001)
r.animation()